In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
re=pd.read_csv('../input/train-label/train_labels.csv')
re

In [ ]:
re=pd.read_csv('../input/train-label/train_labels.csv')
re
re['name']=train['path']
re
re.columns=["path","invasive"]
re
re['invasive']=re['invasive'].astype('str')
re

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_valid=train_test_split(re,test_size=0.2)
x_train

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
idg = ImageDataGenerator(horizontal_flip=True)
train_generator = idg.flow_from_dataframe(x_train, x_col = "path", y_col = "invasive", target_size = (300,300), batch_size = 32)

In [ ]:
idg2=ImageDataGenerator()
vaild_generator=idg2.flow_from_dataframe(x_valid, x_col = "path", y_col = "invasive", target_size = (300,300), batch_size = 32)

In [ ]:
!pip install -U git+https://github.com/qubvel/efficientnet

In [ ]:
from efficientnet.tfkeras import EfficientNetB1
from keras import Sequential
from keras.layers import * 
model=Sequential()
model.add(EfficientNetB1(weights='imagenet',pooling='avg',include_top=False))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
from keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau
es=EarlyStopping(patience=5)
mc=ModelCheckpoint('best.h5',save_best_only=True)
RL=ReduceLROnPlateau(patience=3,factor=0.2) #RL

In [ ]:
model.fit(train_generator,validation_data=vaild_generator,epochs=100,callbacks=[es])


In [ ]:
import pandas as pd
import glob
test=pd.DataFrame({'path':glob.glob('../input/testset/test/*')})
test

In [ ]:
test_generator = idg2.flow_from_dataframe(test, x_col = "path", y_col = None, target_size = (300,300), batch_size = 32, class_mode=None, shuffle=False)

In [ ]:
result=model.predict(test_generator)
result

In [ ]:
sub=pd.read_csv('../input/sample/sample_submission.csv')
sub
sub['name']=test['path'].apply(lambda x:x.split('/')[4].split('.jpg')[0])
sub

In [ ]:
sub["invasive"] = result[:,1]
sub

In [ ]:
sub.to_csv('sub4_2.csv',index=0)